In [3]:
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense
import numpy as np

# Loading the images

In [4]:
import cv2
import os
def load_img(indir):
    samples = []
    labels = []
    for class_dir in os.listdir(indir):
        the_class = class_dir
        for file in os.listdir(indir+'/'+class_dir):
            image = cv2.imread("{}/{}/{}".format(indir,class_dir,file))
            image = cv2.resize(image, (128,128))
            samples.append(image)
            labels.append(the_class)
    samples = np.array(samples)
    labels = np.array(labels)
    return samples,labels
samples, labels = load_img('dataset')
print('loaded',len(samples),' samples')

labels[labels=='new_cats']='cats'
labels[labels=='new_dogs']='dogs'

print('classes',set(labels))

# normalization
samples = samples / 255.0

loaded 6192  samples
classes {'cats', 'dogs'}


# One-hot encoding

In [5]:
from sklearn.preprocessing import LabelEncoder 

encoder = LabelEncoder() # encoder
intlabels = encoder.fit_transform(labels)
labels = tf.keras.utils.to_categorical(intlabels)
print("One-hot labels",labels.shape)

One-hot labels (6192, 2)


In [6]:
import sklearn.model_selection
(trainSamples, testSamples, trainLabels, testLables) = sklearn.model_selection.train_test_split(samples,labels,random_state=42)

# Creating the model

In [7]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D, BatchNormalization

model = Sequential()
model.add(Conv2D(16, (3, 3), padding="same",input_shape=(128,128,3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(32, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(Dropout(0.25))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(Dropout(0.25))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation("relu"))
model.add(Dense(300))
model.add(Activation("relu"))
model.add(Dense(2))
model.add(Activation("softmax"))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 128, 128, 16)      448       
                                                                 
 activation (Activation)     (None, 128, 128, 16)      0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 64, 64, 16)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 64, 64, 32)        4640      
                                                                 
 activation_1 (Activation)   (None, 64, 64, 32)        0         
                                                                 
 dropout (Dropout)           (None, 64, 64, 32)        0         
                                                        

In [8]:
model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])

In [9]:
from tensorflow.python.keras.callbacks import ModelCheckpoint, EarlyStopping

EPOCHS = 30
BATCH_SIZE = 100
checkpt = ModelCheckpoint (filepath='model.h5', save_best_only=True)
estopping = EarlyStopping(monitor='val_loss', patience=3, verbose=1)

model.fit(trainSamples, trainLabels, epochs=EPOCHS, batch_size=BATCH_SIZE, validation_data=(testSamples, testLables), callbacks=[checkpt, estopping])

Epoch 1/30
47/47 [==============================] - 39s 815ms/step - loss: 0.8651 - accuracy: 0.5360 - val_loss: 0.6409 - val_accuracy: 0.6944
Epoch 2/30
47/47 [==============================] - 38s 813ms/step - loss: 0.5619 - accuracy: 0.7117 - val_loss: 0.5199 - val_accuracy: 0.7739
Epoch 3/30
47/47 [==============================] - 42s 891ms/step - loss: 0.4017 - accuracy: 0.8234 - val_loss: 0.4649 - val_accuracy: 0.7804
Epoch 4/30
47/47 [==============================] - 43s 911ms/step - loss: 0.2747 - accuracy: 0.8898 - val_loss: 0.3654 - val_accuracy: 0.8372
Epoch 5/30
47/47 [==============================] - 47s 996ms/step - loss: 0.1688 - accuracy: 0.9309 - val_loss: 0.3371 - val_accuracy: 0.8566
Epoch 6/30
47/47 [==============================] - 49s 1s/step - loss: 0.1005 - accuracy: 0.9632 - val_loss: 0.2608 - val_accuracy: 0.8889
Epoch 7/30
47/47 [==============================] - 50s 1s/step - loss: 0.0857 - accuracy: 0.9660 - val_loss: 0.2320 - val_accuracy: 0.9102
Epoch

# Results

In [10]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, cohen_kappa_score
testResults = model.predict(testSamples)

print(confusion_matrix(testLables.argmax(axis=1), testResults.argmax(axis=1)))
print(classification_report(testLables.argmax(axis=1), testResults.argmax(axis=1)))
print("CNN Accuracy: {:.2f}".format(accuracy_score(testLables.argmax(axis=1), testResults.argmax(axis=1))))
print("Cohen's Kappa: {:.2f}".format(cohen_kappa_score(testLables.argmax(axis=1), testResults.argmax(axis=1))))

49/49 [==============================] - 3s 67ms/step
[[748  29]
 [ 48 723]]
              precision    recall  f1-score   support

           0       0.94      0.96      0.95       777
           1       0.96      0.94      0.95       771

    accuracy                           0.95      1548
   macro avg       0.95      0.95      0.95      1548
weighted avg       0.95      0.95      0.95      1548

CNN Accuracy: 0.95
Cohen's Kappa: 0.90


In [11]:
tf.saved_model.save(model,'model')

INFO:tensorflow:Assets written to: model\assets


INFO:tensorflow:Assets written to: model\assets


In [12]:
from tensorflow.python.keras.models import load_model
model = load_model("final_model.h5") ### add the correct model name!!!
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 128, 128, 16)      448       
_________________________________________________________________
activation_12 (Activation)   (None, 128, 128, 16)      0         
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 64, 64, 16)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 64, 64, 32)        4640      
_________________________________________________________________
activation_13 (Activation)   (None, 64, 64, 32)        0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 64, 64, 32)        0         
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 32, 32, 32)       